# Exercise 8

Last exercise, we want to make an AI that automatically plays FF6 for us. However, due to the complexity of the topic, we aren't able to fully implement everything. Therefore, we created an environment to simulate the game, with simplified game settings. Following are the code to simulate the game, welcome to try :)

In [ ]:
#import libraries
import random

#Equations to calculate HP/MP lost from an attack
def phy_dmg(attack, strength):
    return attack + strength
def m_dmg(spell_pwr, level, magic):
    return spell_pwr * 4 + (level * magic * spell_pwr / 32)
def phy_dmg_reduced(damage, defense):
    return (damage * (255 - defense) / 256) + 1
def m_dmg_reduced(damage, m_def):
    return (damage * (255 - m_def) / 256) + 1
def phy_block_value(evasion):
    return (255 - evasion * 2) + 1
def m_block_value(m_evasion):
    return (255 - m_evasion * 2) + 1
def hit(hit_rate, block_value):
    return int((hit_rate * block_value) / 256 > random.randint(0, 99))

#class
class Ally:
    def __init__(self):
        self.name = "Terra"
        self.level = 3
        self.maxHP = 63
        self.HP = 63
        self.maxMP = 24
        self.MP = 24
        self.strength = 31
        self.speed = 33
        self.stamina = 28
        self.magic = 39
        self.attack = 12
        self.defense = 42
        self.evasion = 0.05
        self.m_defense = 33
        self.m_evasion = 0.07
        self.defend = 0
        self.state = 1

    def attack_action(self, target):
        if hit(target.hit_rate, phy_block_value(self.evasion)):
            target.HP -= int(phy_dmg_reduced(phy_dmg(self.attack, self.strength), target.defense))
            print("Terra attacks", target.name ,"for", int(phy_dmg_reduced(phy_dmg(self.attack, self.strength), target.defense)) ,"damage")
            print("Terra HP:", self.HP, "Target HP:", target.HP)
            return
        print("Attack Missed")

    def cure(self, target):
        print("Terra casts Cure")
        self.MP -= 5
        target.HP += int(m_dmg_reduced(m_dmg(10, self.level, self.magic), target.m_defense))
        print("Terra heals", target.name ,"for", int(m_dmg_reduced(m_dmg(10, self.level, self.magic), target.m_defense)) ,"HP")
        if self.HP >= self.maxHP:
            self.HP = self.maxHP
        print("Terra HP:", self.HP, "Target HP:", target.HP)

    def fire(self, target):
        print("Terra casts Fire")
        self.MP -= 4
        if hit(target.hit_rate, m_block_value(self.m_evasion)):
            target.HP -= int(m_dmg_reduced(m_dmg(21, self.level, self.magic), target.m_defense))
            print("Terra casts Fire and hits", target.name ,"for", int(m_dmg_reduced(m_dmg(21, self.level, self.magic), target.m_defense)),"damage")
            print("Terra HP:", self.HP, "Target HP:", target.HP)
            return
        print("Spell Missed")

    def potion(self, target):
        bag.potion -= 1
        print("Terra uses Potion")
        target.HP += 50
        print("Terra heals", target.name,"for 50 HP")
        if self.HP >= self.maxHP:
            self.HP = self.maxHP
        print("Terra HP:", self.HP)

    def defend_action(self):
        print("Terra uses Defend")
        self.defend = 1

class Foe:
    def __init__(self, enemy):
        self.level = enemy[0]
        self.maxHP = enemy[1]
        self.HP = enemy[1]
        self.maxMP = enemy[2]
        self.MP = enemy[2]
        self.strength = 10
        self.magic = enemy[3]
        self.attack = enemy[4]
        self.defense = enemy[5]
        self.evasion = enemy[6]
        self.m_defense = enemy[7]
        self.m_evasion = enemy[8]
        self.hit_rate = enemy[9]
        self.name = enemy[10]
        self.state = 1

    def attack_action(self, target):
        if self.state == 0:
            return
        elif random.random() < 0.33:
            print(self.name, "did nothing")
            return
        elif hit(100, phy_block_value(self.evasion)):
            target.HP -= int(phy_dmg_reduced(phy_dmg(self.attack, self.strength), target.defense))
            print(self.name, "attacks", target.name ,"for", int(phy_dmg_reduced(phy_dmg(self.attack, self.strength), target.defense)) ,"damage")
            print("Terra HP:", target.HP)
            return
        print(self.name, "Attack Missed")

class Inventory:
    def __init__(self):
        self.potion = 2

class EnemyGroup:
    pass

#stats of enemies in the format [Level, HP, MP, Magic, Attack, Defense, Evasion, Magic Defense, Magic Evasion, Hit Rate]
wererat_stats = [4, 24, 0, 0, 13, 100, 0, 150, 0, 100, "wererat"]
spritzer_stats = [5, 15, 0, 0, 13, 95, 0, 150, 0, 100, "spritzer"]
bandit_stats = [5, 35, 0, 0, 19, 90, 0, 120, 0, 100, "bandit"]

bag = Inventory()
Terra = Ally()
wererat = Foe(wererat_stats)
spritzer = Foe(spritzer_stats)
bandit = Foe(bandit_stats)

#battle sequence
def battle():
    enemies = EnemyGroup()
    enemies.enemy1 = wererat
    enemies.enemy2 = spritzer
    while True:
        if Terra.HP <= 0:
            Terra.state = 0
            break

        if Terra.state == 0 or (enemies.enemy1.state == 0 and enemies.enemy2.state == 0):
            break
        
        print("Terra's Turn")
        Terra.defend = 0
        while True:
            action = input("Please input Attack, Cure, Fire, Potion, or Defend")
            #add AI decision making here
            if action == "Attack":
                target = input("wererat or spritzer")
                if target == "wererat" and enemies.enemy1.state == 1:
                    Terra.attack_action(enemies.enemy1)
                    if enemies.enemy1.HP <= 0:
                        enemies.enemy1.state = 0
                        print(enemies.enemy1.name, "defeated.")
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                elif target == "spritzer" and enemies.enemy2.state == 1:
                    Terra.attack_action(enemies.enemy2)
                    if enemies.enemy2.HP <= 0:
                        enemies.enemy2.state = 0
                        print(enemies.enemy2.name, "defeated.")
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                else:
                    print("Target is Invalid or Dead.")
                    continue
            elif action == "Cure":
                if Terra.MP < 5:
                    print("Not enough MP!")
                    continue
                target = input("Terra? Y or N")
                if target == "Y":
                    Terra.cure(Terra)
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                else:
                    print("Invalid command.")
                    continue
            elif action == "Fire":
                if Terra.MP < 4:
                    print("Not enough MP!")
                    continue
                target = input("wererat or spritzer")
                if target == "wererat" and enemies.enemy1.state == 1:
                    Terra.fire(enemies.enemy1)
                    if enemies.enemy1.HP <= 0:
                        enemies.enemy1.state = 0
                        print(enemies.enemy1.name, "defeated.")
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                elif target == "spritzer" and enemies.enemy2.state == 1:
                    Terra.fire(enemies.enemy2)
                    if enemies.enemy2.HP <= 0:
                        enemies.enemy2.state = 0
                        print(enemies.enemy2.name, "defeated.")
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                else:
                    print("Target is Invalid or Dead.")
                    continue
            elif action == "Potion":
                if bag.potion <= 0:
                    print("No more Potions!")
                    continue
                target = input("Terra? Y or N")
                if target == "Y":
                    Terra.potion(Terra)
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                else:
                    print("Invalid command.")
                    continue
            elif action == "Defend":
                target = input("Terra? Y or N")
                if target == "Y":
                    Terra.defend_action()
                    print("Enemy Turn")
                    if enemies.enemy1.state == 1:
                        enemies.enemy1.attack_action(Terra)
                    if enemies.enemy2.state == 1:
                        enemies.enemy2.attack_action(Terra)
                    break
                else:
                    print("Invalid command.")
                    continue
            print("Invalid command")
        
    print("Battle is Over.")
    if Terra.state == 0:
        print("You Died. Restart?")
    else:
        print("Victory!")
        print("Terra HP:", Terra.HP , "Terra MP:", Terra.MP, "Potion Count:", bag.potion)

battle()


And following are we using the 7 algorithms to find possible operation sequences to win the game
## Note: Please be noted that the code we provided are more like a pseudo code, because we are missing some part to fully make our code functional

# BFS

In [ ]:
from queue import Queue
import copy
answer = [] # Storing answer we found
def BFS():
    actions = [] # Initial Sequence
    q = Queue() # Initializing of BFS
    q.put(actions)
    while q.empty == 0: # Start of BFS
        current = q.get()
        result = battle(current) # Get the result of current sequence
        if result == "Victory!": # If won, we found the answer, no more BFS required
            answer = current
            break
        elif result == "You Died. Restart?" or result == "Invalid command":
            continue # Eliminate impossible sequences
        for acts in ["Attack", "Cure", "Fire", "Potion", "Defend"]: # Go through every possible next action
            temp = copy.deepcopy(current) 
            temp.append(acts)
            if acts == "Attack" or acts == "Fire":
                for target in ["wererat", "spritzer"]: # Selecting target
                    temp2 = copy.deepcopy(temp) 
                    temp2.append(target)
                    q.put(temp2) # Enqueue the sequence for search
            else:
                temp.append("Y")
                q.put(temp) # Enqueue the sequence for search
                
BFS()

# DFS

In [ ]:
from queue import Queue
import copy
answer = []
def DFS(current): # A recursive function for DFS
    result = battle(current) # Get the result of current sequence
    if answer != []: # If we have the answer, there is no more need to search
        return
    if result == "Victory!": # Same here
        answer = current
        return
    elif result == "You Died. Restart?" or result == "Invalid command":
        return # Eliminate impossible sequences
    
    for acts in ["Attack", "Cure", "Fire", "Potion", "Defend"]: # Go through every possible next actions
        temp = copy.deepcopy(current)
        temp.append(acts)
        if acts == "Attack" or acts == "Fire":
            for target in ["wererat", "spritzer"]:
                temp2 = copy.deepcopy(temp)
                DFS(temp2) # Go deeper into DFS
        else:
            temp.append("Y")
            DFS(temp) # Go deeper into DFS
            
DFS([])

# Depth Limited Search

In [ ]:
from queue import Queue
import copy
answer = []
def DFS(current, Limit): # Almost same for DFS, however
    result = battle(current)
    if answer != [] or Limit == 0: # We added a search depth limit here
        return
    if result == "Victory!":
        answer = current
        return
    elif result == "You Died. Restart?" or result == "Invalid command":
        return
    
    for acts in ["Attack", "Cure", "Fire", "Potion", "Defend"]:
        temp = copy.deepcopy(current)
        temp.append(acts)
        if acts == "Attack" or acts == "Fire":
            for target in ["wererat", "spritzer"]:
                temp2 = copy.deepcopy(temp)
                DFS(temp2, Limit - 1)
        else:
            temp.append("Y")
            DFS(temp, Limit - 1)
            
DFS([], 4)

# Iterative Deepening Search

In [ ]:
from queue import Queue
import copy
answer = []
def DFS(current, Limit): # Same for DLS, however
    result = battle(current)
    if answer != [] or Limit == 0:
        return
    if result == "Victory!":
        answer = current
        return
    elif result == "You Died. Restart?" or result == "Invalid command":
        return
    
    for acts in ["Attack", "Cure", "Fire", "Potion", "Defend"]:
        temp = copy.deepcopy(current)
        temp.append(acts)
        if acts == "Attack" or acts == "Fire":
            for target in ["wererat", "spritzer"]:
                temp2 = copy.deepcopy(temp)
                DFS(temp2, Limit - 1)
        else:
            temp.append("Y")
            DFS(temp, Limit - 1)

for i in range(0, 4): # We iterate through depth limit
    DFS([], 4)
    if answer != []:
        break

Uniform Cost Search

In [ ]:
from queue import PriorityQueue 
import copy
answer = [] # Storing answer we found

def cost(player, enemies, bag): # Cost Function
    enemy1 = enemies.enemy1
    enemy2 = enemies.enemy2
    return (max(enemy1.HP, 0) + max(enemy1.HP, 0)) * ((enemy1.state + enemy2.state)**3) - player.state * (player.HP + 10 * player.MP + 50 * bag.potion)


def UCS():
    cost = 0
    actions = [] # Initial Sequence
    q = PriorityQueue() # Initializing of UCS
    player, enemies, bag, result = battle(current)
    q.put((cost(player, enemies, bag), actions))
    while q.empty == 0: # Start of UCS
        current = q.get()
        if answer != []:
            break
        for acts in ["Attack", "Cure", "Fire", "Potion", "Defend"]: # Go through every possible next action
            temp = copy.deepcopy(current) 
            temp.append(acts)
            if acts == "Attack" or acts == "Fire":
                for target in ["wererat", "spritzer"]: # Selecting target
                    temp2 = copy.deepcopy(temp) 
                    temp2.append(target)
                    player, enemies, bag, result = battle(temp2) # Get info for calculating cost
                    if result == "Victory!": # If won, we found the answer, no more UCS required
                        answer = temp2
                        break
                    elif result == "You Died. Restart?" or result == "Invalid command":
                        continue # Eliminate impossible sequences
                    q.put((cost(player, enemies, bag), temp2)) # Put the tuple (cost, sequence) into priority queue
            else:
                temp.append("Y")
                player, enemies, bag, result = battle(temp) # Get info for calculating cost
                if result == "Victory!": # If won, we found the answer, no more UCS required
                    answer = temp
                    break
                elif result == "You Died. Restart?" or result == "Invalid command":
                    continue # Eliminate impossible sequences
                q.put((cost(player, enemies, bag), temp)) # Put the tuple (cost, sequence) into priority queue
                
UCS()

# Greedy

# A* Algorithm